In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tqdm
from matplotlib.animation import FuncAnimation, PillowWriter

In [ ]:
# Define parameters (set T_A to any desired quantity, ideally lower than T_C)
N = 100         # Lattice size (NxN grid)
J = 1          # Interaction strength
k_B = 1        # Boltzmann constant
STEPS = 500_000  # Total MCMC steps
BURNIN = 50_000  # Burn-in period
T_A = 2        # Temperature

In [ ]:
# Define the range of B values (forward and reverse sweep)
B_vals = np.linspace(-0.4, 0.4, 24)
B_vals = np.concatenate([B_vals, B_vals[::-1]])  # Sweep B forward & back

In [ ]:
# Initialize lattice
def initialize_lattice(N, mode="ordered"):
    if mode == "ordered":
        return np.ones((N, N))
    elif mode == "random":
        return np.random.choice([-1, 1], size=(N, N))

In [ ]:
# Metropolis Algorithm
def metropolis(lattice, T, B, steps):
    N = lattice.shape[0]
    magnetization = []
    num_accept = 0

    for _ in tqdm.tqdm(range(steps), leave=False):
        i, j = np.random.randint(N), np.random.randint(N)  # Pick random spin
        delta_E = 0

        for k, l in [(-1, 0), (1, 0), (0, 1), (0, -1)]:  # Nearest neighbors
            i_neigh = (i + k) % N
            j_neigh = (j + l) % N
            delta_E += -J * -2 * lattice[i, j] * lattice[i_neigh, j_neigh] - B * lattice[i, j]

        if delta_E <= 0 or np.random.random() < np.exp(-delta_E / (k_B * T)):
            lattice[i, j] *= -1  # Flip spin
            num_accept += 1

        magnetization.append(np.mean(lattice))

    return magnetization


In [ ]:
# Initialize lattice
lattice_ordered = initialize_lattice(N, mode="random")

# Run simulation for each B value and store magnetization curves
magnetization_data = []
for B_field in B_vals:
    m_values = metropolis(lattice_ordered, T_A, B_field, STEPS)
    magnetization_data.append(m_values)

In [ ]:
# Create figure for animation
fig, ax = plt.subplots(figsize=(8, 6))
line, = ax.plot([], [], lw=2)
ax.axvline(BURNIN, color="black", linestyle="--", label="Burn-in threshold")
ax.set_xlabel("MCMC Steps")
ax.set_ylabel("Net Magnetization")
ax.set_ylim(-1.1, 1.1)
ax.set_xlim(0, STEPS)
ax.legend()
title = ax.set_title(f"Magnetization vs. Steps (B = {B_vals[0]:.2f})")

In [ ]:
# Animation update function
def update(frame):
    B_field = B_vals[frame]
    line.set_data(range(STEPS), magnetization_data[frame])
    title.set_text(f"Magnetization vs. Steps (B = {B_field:.2f})")
    return line, title

In [ ]:
# Create animation
ani = FuncAnimation(fig, update, frames=len(B_vals), interval=200)

# Save as GIF using Pillow
ani.save("magnetization_vs_B.gif", writer=PillowWriter(fps=5))

print("Animation saved as magnetization_vs_B.gif")
plt.show()